In [ ]:
!pip install opencv-contrib-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Question 1

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
original_image = cv2.imread("/content/touching_grayscale.png")
cv2_imshow(original_image)
# print(original_image.shape)

In [ ]:
img = cv2.imread('/content/touching_grayscale.png', cv2.IMREAD_GRAYSCALE)
cv2_imshow(img)

In [ ]:
pixels_vals = original_image.reshape((-1, 3))
pixel_vals = np.float32(pixels_vals)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 10 #check at K = 3 and K = 5
attempts = 10
ret, label, center = cv2.kmeans(pixel_vals, K, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)
center = np.uint8(center)
res = center[label.flatten()]
result_image = res.reshape((original_image.shape))
figure_size=15
plt.figure(figsize=(figure_size, figure_size))
plt.subplot(1,2,1), plt.imshow(original_image)
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(1,2,2), plt.imshow(result_image)
plt.title('Segmented Image when K = %i' %K), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
resultimage1 = cv2.cvtColor(result_image, cv2.COLOR_BGR2GRAY)
otsu_threshold, image_result = cv2.threshold(resultimage1, 200, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
print("Obtained threshold: ", otsu_threshold)
contours, hierarchy = cv2.findContours(image_result, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for c in contours:
  M = cv2.moments(c)
  if M["m00"] != 0:
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
  else:
    cX, cY = 0, 0
  cv2.circle(resultimage1, (cX, cY), 1, (255, 0, 0), -1)
cv2_imshow(resultimage1)
cv2.drawContours(image_result, contours, -1, (0, 255, 0), 3)
cv2_imshow(image_result)

## Question 2

1)

In [ ]:
'''
image_shape=size(image) 
images= Array [ ] 
for size in 1:5 
resized_image=image.reshape(image_shape//size) 
insert resized_image into images 
end
blurred_images=[ ] 
sigmas=Array[] # predecided values of sigma for example from 0.1,1.5,3.0,4.5,6.0 
for i in 1:5 
scale=Array [] 
for sigma in sigmas 
insert GauassianBlur(images[ i ], sigma) into scale 
end 
insert scale into blurred_images
end 
DOG=Array [] for i in blurred_images 

scale=Array [] 
for index in 1:length(i)-1 
insert (i[index]-i[index+1]) into Scale 
end 
insert scale into DOG 
keypoint= Array [] 
for each scale_image in DOG 
for each image in scale_image 
for each pixel in image 
if image[pixel]>image[8-neighbourhood] and image[pixel] > other images[pixel] in sam 
insert pixel into keypoint
remove keypoints if near an edge or if keypoint is a low contrast pixel orientation=[] 
for keypoint in keypoints: 
a neighbourhood of pixels are selected around the keypoint gradient_magnitude,gradient_direction are found in that region all points above 80% of max maginitude are selected and considered to calculate orientation, 
add such points on orientation subtract localised orientation to make it rotation independent.'''

'\nimage_shape=size(image) \nimages= Array [ ] \nfor size in 1:5 \nresized_image=image.reshape(image_shape//size) \ninsert resized_image into images \nend\nblurred_images=[ ] \nsigmas=Array[] # predecided values of sigma for example from 0.1,1.5,3.0,4.5,6.0 \nfor i in 1:5 \nscale=Array [] \nfor sigma in sigmas \ninsert GauassianBlur(images[ i ], sigma) into scale \nend \ninsert scale into blurred_images\nend \nDOG=Array [] for i in blurred_images \n\nscale=Array [] \nfor index in 1:length(i)-1 \ninsert (i[index]-i[index+1]) into Scale \nend \ninsert scale into DOG \nkeypoint= Array [] \nfor each scale_image in DOG \nfor each image in scale_image \nfor each pixel in image \nif image[pixel]>image[8-neighbourhood] and image[pixel] > other images[pixel] in sam \ninsert pixel into keypoint\nremove keypoints if near an edge or if keypoint is a low contrast pixel orientation=[] \nfor keypoint in keypoints: \na neighbourhood of pixels are selected around the keypoint gradient_magnitude,gradien




SIFT is invariant to scale and orientation, making it possible to extract the same features from images taken from different viewpoints of an object, SIFT extracts those features where are scale and rotation invariant and these keypoints can be used as features for object detection , etc.

2)

In [ ]:
img1 = cv2.imread('/content/monument1.jpeg')
img2 = cv2.imread('/content/monument2.jpeg')
def doHarris(img):
  grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  grayscale = np.float32(grayscale)
  dst = cv2.cornerHarris(grayscale, 2, 3, 0.04)
  dst = cv2.dilate(dst, None)
  img[dst>0.01*dst.max()] = [0,0,255]
  return img

In [ ]:
cv2_imshow(doHarris(img1))

In [ ]:
cv2_imshow(doHarris(img2))

In [ ]:
query_img_bw = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
train_img_bw = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
orb = cv2.ORB_create()
queryKeypoints, queryDescriptors = orb.detectAndCompute(query_img_bw, None)
trainKeypoints, trainDescriptors = orb.detectAndCompute(train_img_bw, None)
matcher = cv2.BFMatcher()
matches = matcher.match(queryDescriptors, trainDescriptors)
final_img = cv2.drawMatches(img1, queryKeypoints, img2, trainKeypoints, matches[:20], None)
final_img = cv2.resize(final_img, (1000, 650))
cv2_imshow(final_img)

In [ ]:
query_img_bw = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
train_img_bw = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
orb = cv2.ORB_create()
queryKeypoints, queryDescriptors = orb.detectAndCompute(query_img_bw, None)
trainKeypoints, trainDescriptors = orb.detectAndCompute(train_img_bw, None)
matcher = cv2.BFMatcher()
matches = matcher.match(queryDescriptors, trainDescriptors)
final_img = cv2.drawMatches(img1, queryKeypoints, img2, trainKeypoints, matches[:20], None)
final_img = cv2.resize(final_img, (1000, 650))
cv2_imshow(final_img)

Harris corner detection algorithm identifies the internal corners of an image. The corners of an image are basically identified as the regions in which there are variations in large intensity of the gradient in all possible dimensions and directions.
Harris corner detection works as:
1. Take the grayscale image of the original one
2. Apply Sobel operator to find the x and y gradient values for every pixel in the grayscale image
3. For each pixel p in the grayscale image, consider a 3x3 window around it and compute the corner strength function.  Call this its Harris value.
4. Find all pixels that exceed a certain threshold and are the local maxima within a certain window.
5. For each pixel that meets the criteria in 4, compute a feature descriptor


ORB use both the FAST keypoint detector and BRIEF descriptor with some added features to improve the performance. FAST is Features from Accelerated Segment Test used to detect features from the provided image.
ORB uses BRIEF descriptor but as the BRIEF performs poorly with rotation. ORB is an efficient alternative to SIFT or SURF algorithms used for feature extraction, in computation cost, matching performance and mainly the patents.
We have used the following algorithm:
1. Take the input image and convert it into grayscale
2. Initialize the ORB detector and detect the keypoints in the input image
3. Now we have computed the descriptors belonging to both the images
4. Then, the keypoints were matched using the Brute Force matcher
5. Show the matched images
For this question, we have used the image from two different perspectives.

## Question 3

In [ ]:
pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jessicali9530/lfw-dataset


In [2]:
!kaggle datasets download -d ciplab/real-and-fake-face-detection

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip real-and-fake-face-detection.zip

In [ ]:
!unzip lfw-dataset.zip

In [ ]:
original = cv2.imread('/content/lfw-deepfunneled/lfw-deepfunneled/Zico/Zico_0002.jpg')
cv2_imshow(original)

In [ ]:
grayscale_image = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
detected_faces = face_cascade.detectMultiScale(grayscale_image)
for (column, row, width, height) in detected_faces:
  cv2.rectangle(original, (column, row),(column + width, row + height), (0, 255, 0), 2)
plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
import dlib
img6 = cv2.imread('/content/lfw-deepfunneled/lfw-deepfunneled/Zico/Zico_0002.jpg')
gray = cv2.cvtColor(img6, cv2.COLOR_BGR2GRAY)
hogFaceDetector = dlib.get_frontal_face_detector()
faces = hogFaceDetector(gray, 1)
for (i, rect) in enumerate(faces):
  x = rect.left()
  y = rect.top()
  w = rect.right() - x
  h = rect.bottom() - y
  cv2.rectangle(img6, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2_imshow(img6)

## Question 4

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame()
Word="The Car Truck is Driven on Road Highway".split(" ")
A = [2/7, 1/7, 0, 1/7, 1/7,  1/7, 1/7, 0]
B = [2/7, 0, 1/7, 1/7, 1/7,  1/7, 0, 1/7]

df['Word']=Word
df['A'] = A
df['B'] = B
df

In [ ]:
idf = []

In [ ]:
for index, row in df.iterrows():
    val = 0
    if row['A']>0:
        val+=1
    if row['B']>0:
        val+=1
    idf.append(np.log(2/val))
df['idf']=idf
df['tfidf_A'] = df['A']*df['idf']
df['tfidf_B'] = df['B']*df['idf']

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
plt.bar(df['Word'],df['tfidf_A'])

In [ ]:
plt.bar(df['Word'],df['tfidf_B'])

In [ ]:
dataset = ['The car is driven on the road','The truck is driven on the highway']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer

tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(dataset)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
print (df)

In [ ]:
S1 = 'The car is driven on the road'
S2 = 'The truck is driven on the highway'

In [ ]:
vectorizer = TfidfVectorizer(use_idf=True)
response = vectorizer.fit_transform([S1,S2])

In [ ]:
print(response)

## Question 5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)
lk_params = dict(winSize = (15, 15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
cap = cv2.VideoCapture("/content/drive/MyDrive/video.mp4")
color = (0,255,0)
ret, first_frame = cap.read()
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
prev = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)
mask = np.zeros_like(first_frame)
while (cap.isOpened()):
  ret, frame = cap.read()
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  prev = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
  next, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev, None, **lk_params)
  good_old = prev[status == 1].astype(int)
  good_new = next[status == 1].astype(int)
  for i, (new, old) in enumerate(zip(good_new, good_old)):
    a, b = new.ravel()
    c, d = old.ravel()
    mask = cv2.line(mask, (a, b), (c, d), color, 2)
    frame = cv2.circle(frame, (a, b), 3, color, -1)
  output = cv2.add(frame, mask)
  prev_gray = gray.copy()
  prev = good_new.reshape(-1 , 2)
  cv2_imshow(output)

In [ ]:
cap = cv2.VideoCapture("/content/drive/MyDrive/video.mp4")
ret, first_frame = cap.read()
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(first_frame)
mask[...,1] = 255
while(cap.isOpened()):
  ret, frame = cap.read()
  cv2_imshow(frame)
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
  magnitude, angle = cv2.cartToPolar(flow[...,0], flow[...,1])
  mask[...,0] = angle * 180 / np.pi / 2
  mask[...,2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
  rgb = cv2. cvtColor(mask, cv2.COLOR_HSV2BGR)
  cv2_imshow(rgb)
  prev_gray = gray

## Question 6

In [ ]:
from skimage import feature
import glob
images = []
labels = []
path = "/content/real_and_fake_face/training_real/*.*"
for file in glob.glob(path):
  img6 = cv2.imread(file)
  img6 = cv2.resize(img6, (128, 256))
  hog_desc = feature.hog(img6, orientations=9, pixels_per_cell=(8, 8),
                         cells_per_block=(2, 2), 
                         transform_sqrt=True, block_norm='L2-Hys')
  images.append(hog_desc)
  labels.append(1)
# print(labels)

In [ ]:
path = "/content/real_and_fake_face/training_fake/*.*"
for file in glob.glob(path):
  img = cv2.imread(file)
  img = cv2.resize(img, (128, 256))
  hog_desc = feature.hog(img, orientations=9, pixels_per_cell=(8, 8), 
                        cells_per_block=(2,2), transform_sqrt=True,
                         block_norm='L2-Hys')
  images.append(hog_desc)
  labels.append(0)

In [ ]:
print(len(images))

2041


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=42)

In [ ]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC(random_state = 42, tol=1e-5)
clf = svm_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
predictions = svm_model.predict(X_test)
print(accuracy_score(y_test, predictions)*100)

54.01174168297456


##Question 7 
part one 

In this approach, we have applied a function ‘hog_detect’ for evaluating the hog descriptions of each image of the handwritten digits datasets and saved the output of each descriptions (which is an array) into an array which works as the feature set for the SVM model. We have applied SVC and applied polynomial kernel as the datasets are divided into multiple labels rather than being binary labels

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.preprocessing import normalize

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]
# define the larger model
def larger_model():
	# create model
	model = Sequential()
	model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
	model.add(MaxPooling2D())
	model.add(Conv2D(15, (3, 3), activation='relu'))
	model.add(MaxPooling2D())
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# build the model
model = larger_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)


Histogram of Oriented Gradients (HOG) is a feature descriptor that helps us finding the explicit and robust local features of an image. A feature descriptor is a representation of an image or an image patch that is a simplified version obtained by extracting useful information and throwing away extraneous information from the original image. In the HOG feature descriptor, the distribution (histograms) of directions of gradients (oriented gradients) are used as features. Gradients (x and y derivatives) of an image are useful because the magnitude of gradients is large around edges/corners which contain abrupt changes in intensity which contains lot of information regarding the object shape as compared to flatter regions. In HOG,feature extraction is performed by first dividng the complete image into smaller regions and calculating the gradients and orientation for each of these regions. This is followed by normalization over blocks of cells to reduce lighting variation since gradients of image are sensitive to overall lighting.

part two

In [ ]:
import cv2
import numpy as np
from keras.datasets import mnist
from matplotlib import pyplot 
from skimage.transform import resize
import math
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score
import pickle
#loading
(train_X, train_y), (test_X, test_y) = mnist.load_data()

train_X = np.array(~train_X[:50000] , dtype=float)
train_y = np.array(train_y[:50000])
test_img = np.array(~test_X[:10000] , dtype=float)
test_label = np.array(test_y[:10000])

training_img = np.pad(train_X, ((0,0), (2,2), (2, 2)), 'constant')
test_img = np.pad(test_img, ((0,0), (2,2), (2, 2)), 'constant')

training_gradiant_x = np.pad(training_img, ((0,0), (1,1), (1, 1)), 'constant')
training_gradiant_y = np.pad(training_img, ((0,0), (1,1), (1, 1)), 'constant')

test_gradiant_x = np.pad(test_img, ((0,0), (1,1), (1, 1)), 'constant')
test_gradiant_y = np.pad(test_img, ((0,0), (1,1), (1, 1)), 'constant')

training_Magnitude = np.zeros((50000,32,32))
training_Angle = np.zeros((50000,32,32))

test_Magnitude = np.zeros((50000,32,32))
test_Angle = np.zeros((50000,32,32))

training_feature_vector = np.zeros((50000,9,4,9))
test_feature_vector = np.zeros((10000,9,4,9))

def calc_Gradiant_x(x,images_array):
    
    for i in range(x):
        for j in range(1,34):
            for k in range(1,33):
                images_array[i][j][k] = images_array[i][j][k+1] - images_array[i][j][k-1] / 2
    
    return images_array

def calc_Gradiant_y(x,images_array):
    
    for i in range(x):
        for j in range(1,33):
            for k in range(1,34):
                images_array[i][j][k] = images_array[i][j+1][k] - images_array[i][j-1][k]  / 2
    
    return images_array

def calc_Magnitude(x,gradiant_x,gradiant_y):
    images_array = np.zeros((x,32,32))
    for i in range(x):
        for j in range(0,32):
            for k in range(0,32):
                images_array[i][j][k] = math.sqrt( pow(gradiant_x[i][j+1][k+1],2) + pow(gradiant_y[i][j+1][k+1],2) )
    return images_array   

def calc_Angle(x,gradiant_x,gradiant_y):
    images_array = np.zeros((x,32,32))
    for i in range(x):
        for j in range(0,32):
            for k in range(0,32):
                images_array[i][j][k] = np.rad2deg(np.arctan(gradiant_y[i][j+1][k+1] / (gradiant_x[i][j+1][k+1]+0.0000001)))
                images_array[i][j][k] = math.ceil(images_array[i][j][k]%180)
    return images_array            

def calc_feature_vector(x,training_Magnitude,training_Angle):
    images_array =np.zeros((x,9,4,9))
    
    
    for pic in range(x):
        c1 = 0  
        c2 = 0
        for block in range(9):
            if block == 1 :
                c1=0
                c2=8
            elif block == 2:
                c1=0
                c2=16
            elif block == 3:
                c1 =8
                c2 = 0
            elif block == 4: 
                c1=8
                c2=8
            elif block == 5:
                c1=8
                c2=16
            elif block == 6:
                c1=16 
                c2=0 
            elif block == 7:
                c1=16
                c2=8
            elif block == 8:
                c1=16
                c2=16
            for cell in range(4):
                
                if cell == 1 :
                    c2+=8
                elif cell == 2:
                    c1+=8
                    c2-=8
                elif cell == 3:
                    c2+=8
                for i in range(c1,c1+8):
                    for j in range(c2,c2+8):
                        print(str(pic))
                        x = training_Angle[pic][i][j]
                        if x==180:
                            images_array[pic][block][cell][8] = training_Magnitude[pic][i][j]
                            continue
                        p1 = (x - math.ceil(x/20)*20 - 10) / ((math.floor(x/20)+1)*20)
                        p2 = (x - math.ceil(x/20)*20 + 10) / ((math.floor(x/20)+1)*20)
                        
                        if p1 <= 0 :
                            images_array[pic][block][cell][math.floor(x/20)] += training_Magnitude[pic][i][j]
                            
                        else :
                            images_array[pic][block][cell][math.floor(x/20)] += p2 * training_Magnitude[pic][i][j]
                            images_array[pic][block][cell][math.floor(x/20) + 1] += p1 * training_Magnitude[pic][i][j]
                  
    return images_array 

def normalization(x,images_array):
    temp = []
    for pic in range(x):
        for block in range(9):
            Sum = np.sum(images_array[pic][block]) 
            if Sum == 0:
                continue
            for i in range(4):
                for j in range(9):
                    images_array[pic][block][i][j] = images_array[pic][block][i][j] / Sum
        temp.append(images_array[pic].flatten().tolist())         
    
    return temp     

def training(training_img):      
    calc_Gradiant_x(10000,training_gradiant_x)
    calc_Gradiant_y(10000,training_gradiant_y)
    training_Magnitude = calc_Magnitude(10000,training_gradiant_x,training_gradiant_y)
    training_Angle = calc_Angle(10000,training_gradiant_x,training_gradiant_y)
    training_feature_vector = calc_feature_vector(10000,training_Magnitude,training_Angle)
    training_fv = normalization(10000,training_feature_vector)

    np.save('training_fv.txt',training_fv)
    return training_fv

def test(test_img):      
    calc_Gradiant_x(3000,test_gradiant_x)
    calc_Gradiant_y(3000,test_gradiant_y)
    test_Magnitude = calc_Magnitude(3000,test_gradiant_x,test_gradiant_y)
    test_Angle = calc_Angle(3000,test_gradiant_x,test_gradiant_y)
    test_feature_vector = calc_feature_vector(3000,test_Magnitude,test_Angle)
    test_fv = normalization(3000,test_feature_vector)
    np.save('test_fv.txt',test_fv)
    return test_fv

def fo(training_fv,train_y,test_fv,test_label):
    training_fv = np.asarray(training_fv , dtype='float')
    test_fv = np.asarray(test_fv ,dtype = 'float')
    

    clf = svm.SVC()
    clf.fit(training_fv,train_y)
    
    pred = clf.predict(test_fv)

    filename = 'SvMClass.sav'
    pickle.dump(clf, open(filename, 'wb'))

    print("Accurate percentage: " + str(accuracy_score(test_label, pred)))
    print('\n')
    print(classification_report(test_label, pred))
    

def main():
    training_fv = training(training_img)
    print("train")
    test_fv = test(test_img)
    print("test")
    fo(training_fv,train_y[:10000],test_fv,test_label[:3000])
    
main()

In [ ]:
df_result = pd.DataFrame(y_pred, columns=['Label'])
df_result.insert(0, 'ImageId', range(1, 1 + len(y_pred)))
df_result.to_csv('result.csv', index=False)
df_result.head()